In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("spark_sql_basic1")
sc   = SparkContext(conf=conf)

24/12/10 14:38:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/10 14:38:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark_sql_basic").getOrCreate()

In [5]:
movies_rdd = sc.parallelize([
    (1, ("어벤져스", "마블")),
    (2, ("슈퍼맨", "DC")),
    (3, ("배트맨", "DC")),
    (4, ("겨울왕국", "디즈니")),
    (5, ("아이언맨", "마블"))
])


attendances_rdd = sc.parallelize([
    (1, (13934592, "KR")),
    (2, (2182227,"KR")),
    (3, (4226242, "KR")),
    (4, (10303058, "KR")),
    (5, (4300365, "KR"))
])

In [6]:
# 관객수가 500만 이상인 영화 가져오기
1. 조인 > 필터 > 영화정보
2. 필터 > 조인 > 영화정보

SyntaxError: invalid syntax (3221445172.py, line 2)

In [7]:
# 조인 먼저

movies_att = movies_rdd.join(attendances_rdd)
movies_att.filter(
    lambda x: x[1][1][0] > 5000000
).collect()

[(4, (('겨울왕국', '디즈니'), (10303058, 'KR'))),
 (1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [8]:
# 필터 먼저
filtered_att = attendances_rdd.filter(lambda x: x[1][0] >5000000)
movies_rdd.join(filtered_att).collect()

[(4, (('겨울왕국', '디즈니'), (10303058, 'KR'))),
 (1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [9]:
movies = [
    (1, "어벤져스", "마블", 2012, 4, 26),
    (2, "슈퍼맨", "DC", 2013, 6, 13),
    (3, "배트맨", "DC", 2008, 8, 6),
    (4, "겨울왕국", "디즈니", 2014, 1, 16),
    (5, "아이언맨", "마블", 2008, 4, 30)
]

In [10]:
movie_schema = ['id', 'name', 'company', 'year', 'month', 'day']

# 1. DataFrame 만들기

In [11]:
df = spark.createDataFrame(data=movies, schema=movie_schema)

In [12]:
df.select('name').show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [13]:
df.filter(df.year>=2000).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



## SQL 사용 위해 View 등록

In [14]:
df.createOrReplacmovies = [
    (1, "어벤져스", "마블", 2012, 4, 26),
    (2, "슈퍼맨", "DC", 2013, 6, 13),
    (3, "배트맨", "DC", 2008, 8, 6),
    (4, "겨울왕국", "디즈니", 2014, 1, 16),
    (5, "아이언맨", "마블", 2008, 4, 30)
]

In [15]:
df.createOrReplaceTempView('movies')

In [16]:
query = '''
select name from movies
'''

spark.sql(query).show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [17]:
# 2010년 이후 개봉한 영화 조회

query = '''
select name, company
from movies
where year>=2010
'''

In [18]:
spark.sql(query).show()

+--------+-------+
|    name|company|
+--------+-------+
|어벤져스|   마블|
|  슈퍼맨|     DC|
|겨울왕국| 디즈니|
+--------+-------+



In [19]:
# 회사가 마블인 영화 목록

query = '''
select name, company
from movies
where company='마블'
'''

spark.sql(query).show()

+--------+-------+
|    name|company|
+--------+-------+
|어벤져스|   마블|
|아이언맨|   마블|
+--------+-------+



In [20]:
# 맨으로 끝나는 영화

query = '''
select name, company
from movies
where name like '%맨'
'''

spark.sql(query).show()

+--------+-------+
|    name|company|
+--------+-------+
|  슈퍼맨|     DC|
|  배트맨|     DC|
|아이언맨|   마블|
+--------+-------+



In [21]:
# '아'로 시작하는 영화

query = '''
select name, company
from movies
where name like '아%'
'''

spark.sql(query).show()

+--------+-------+
|    name|company|
+--------+-------+
|아이언맨|   마블|
+--------+-------+



In [22]:
# 개봉월이 4-8월 사이

query = '''
select name, company
from movies
where month between 4 and 8
'''

spark.sql(query).show()

+--------+-------+
|    name|company|
+--------+-------+
|어벤져스|   마블|
|  슈퍼맨|     DC|
|  배트맨|     DC|
|아이언맨|   마블|
+--------+-------+



In [23]:
# and, or
# ~맨, 개봉연도 2010년 이하

query = '''
select name, company
from movies
where name like '%맨' and year <= 2010
'''

spark.sql(query).show()

+--------+-------+
|    name|company|
+--------+-------+
|  배트맨|     DC|
|아이언맨|   마블|
+--------+-------+



In [61]:
# 회사가 마블,dc

query = '''
select name, company
from movies
where company = '마블' or company ='DC'
'''

spark.sql(query).show()

+--------+-------+
|    name|company|
+--------+-------+
|어벤져스|   마블|
|  슈퍼맨|     DC|
|  배트맨|     DC|
|아이언맨|   마블|
+--------+-------+



In [62]:
query = '''
select name, company
from movies
order by year ASC
'''

spark.sql(query).show()

+--------+-------+
|    name|company|
+--------+-------+
|  배트맨|     DC|
|아이언맨|   마블|
|어벤져스|   마블|
|  슈퍼맨|     DC|
|겨울왕국| 디즈니|
+--------+-------+



In [63]:
query = '''
select *
from movies
'''

spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [42]:
attendances = [
    [1, 13934592., "KR"],
    (2, 2182227.,"KR"),
    (3, 4226242., "KR"),
    (4, 10303058., "KR"),
    (5, 4300365., "KR")
]

In [43]:
from pyspark.sql.types import StringType, IntegerType, FloatType, StructType, StructField

att_schema = StructType([ # 모든 컬럼의 타입을 통칭 - 컬럼 데이터의 집합
    StructField("id", IntegerType(), True), # StructField : 컬럼
    StructField("att", FloatType(), True),
    StructField("theater_country", StringType(), True)
])

In [44]:
att_df = spark.createDataFrame( data=attendances, schema = att_schema )
att_df.dtypes

[('id', 'int'), ('att', 'float'), ('theater_country', 'string')]

In [54]:
att_df.createOrReplaceTempView('att')

In [55]:
att_df.select('*').show()

+---+-----------+---------------+
| id|        att|theater_country|
+---+-----------+---------------+
|  1|1.3934592E7|             KR|
|  2|  2182227.0|             KR|
|  3|  4226242.0|             KR|
|  4|1.0303058E7|             KR|
|  5|  4300365.0|             KR|
+---+-----------+---------------+



In [56]:
query = '''
select movies.id, movies.name, movies.company, att.att, att.theater_country
from movies JOIN att ON  movies.id = att.id
'''

In [64]:
movies_views = spark.sql(query)

In [65]:
movies_views.createOrReplaceTempView('movies_view')

In [66]:
query = '''
select * from movies_view
'''

In [67]:
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [68]:
query = '''
select * 
from movies_view
where company =
'''